In [1]:
%reload_ext autoreload
%autoreload 2

from _header_data import *

/opt/conda/envs/dev/lib/python3.12/site-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


---
### Parallel Streaming and Processing
uses n-1 threads for downloading and processing\
is broken: has to be updated to new processing pipeline

In [ ]:
"""
Load Huggingface dataset for streaming and reload download state
"""

dataset_mdcath = load_dataset("compsciencelab/mdCATH", split="train", streaming=True)
dataset_mdcath = dataset_mdcath.cast_column("image", Image(decode=False))

path_state_dict = "../tmp/data/3Di/_state.pkl"

if os.path.exists(path_state_dict):
    with open(path_state_dict, "rb") as f:
        state = pickle.load(f)
    print("State loaded,", state)
    dataset_mdcath = dataset_mdcath.skip(state)
else:
    state = 0
    print("No state found", state)

In [ ]:
"""
This script processes a dataset using multiprocessing to parallelize the workload.
"""

print("CPU Count: ", multiprocessing.cpu_count())

iterations = 1
urls = [item["image"]["path"] for item in dataset_mdcath.take(iterations)]

pool = multiprocessing.Pool(processes=multiprocessing.cpu_count() - 1 or 1)

dataset_mdcath_mapped = pool.map(
    # partial(download_process_pipeline, config=PROCESSING_CONFIG),
    download_process_pipeline,
    urls
)
pool.close()
pool.join()

In [ ]:
state = state + iterations
with open(path_state_dict, "wb") as f:
    pickle.dump(state, f)
print(dataset_mdcath.state_dict())
print("State saved", state)

---
### Process from Downloaded

In [9]:
# if os.path.exists(os.path.join(FILE_PATHS['pssm'], 'dict_pssm.npy')):
#     dict_pssm = np.load(os.path.join(FILE_PATHS['pssm'], 'dict_pssm.npy'), allow_pickle=True).item()
# else:
dict_pssm = {}

for file in os.listdir(FILE_PATHS['trajectories'])[:2]:
    processed_traj = download_process_pipeline(
        path=f"{FILE_PATHS['trajectories']}/{file}",
        # url='hf://datasets/compsciencelab/mdCATH@2393a6d9aebd04b688022d7a681b7c96d608de79/data/mdcath_dataset_1a2nA02.h5',
        # config=PROCESSING_CONFIG,
        # config=None,
        # save_path_pdb=FILE_PATHS['PDB'],
        save_path_fasta=FILE_PATHS['3Di'],
    )
    dict_pssm.update(processed_traj)
    
    
np.save(os.path.join(FILE_PATHS['pssm'], 'dict_pssm.npy'), dict_pssm)

/Users/finnlueth/Developer/gits/bachelor-thesis/mdcath-to-3di/.venv/lib/python3.12/site-packages/Bio/PDB/PDBParser.py:384: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 1013
  warnings.warn(
/Users/finnlueth/Developer/gits/bachelor-thesis/mdcath-to-3di/.venv/lib/python3.12/site-packages/MDAnalysis/coordinates/memory.py:535: DeprecationWarning: asel argument to timeseries will be renamed to'atomgroup' in 3.0, see #3911
  warnings.warn(
/Users/finnlueth/Developer/gits/bachelor-thesis/mdcath-to-3di/src/data/mdcath_processing.py:385: RuntimeWarning: divide by zero encountered in log2
  pssm = np.log2(pwm)
/Users/finnlueth/Developer/gits/bachelor-thesis/mdcath-to-3di/.venv/lib/python3.12/site-packages/Bio/PDB/PDBParser.py:384: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 6203
  warnings.warn(
/Users/finnlueth/Developer/gits/bachelor-thesis/mdcath-to-3di/.venv/lib/python3.12/site-packages/MDAnalysis/coordinates/memory.py:535: DeprecationWarning: ase

In [ ]:
print(list(dict_pssm.keys()))
display(pd.DataFrame(dict_pssm[list(dict_pssm.keys())[0]]))
print((sys.getsizeof(dict_pssm[list(dict_pssm.keys())[0]])*5*5*5000)/(1024**3))